In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
#Sophie changed to version 3.4.2 so it would run on colab
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,080 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,048 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securi

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

homesales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [4]:
homesales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
homesales_df.createOrReplaceTempView('homesales')


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT
YEAR(date) Year,
ROUND(AVG(price),2) Price
FROM homesales
WHERE bedrooms = 4
GROUP BY Year
""").show()


+----+---------+
|Year|    Price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT
YEAR(date) Year,
ROUND(AVG(price),2) Price
FROM homesales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY Year
""").show()


+----+---------+
|Year|    Price|
+----+---------+
|2022|292725.69|
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
+----+---------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT
YEAR(date) Year,
ROUND(AVG(price),2) Price
FROM homesales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living = 2000 OR sqft_living > 2000
GROUP BY Year
""").show()


+----+---------+
|Year|    Price|
+----+---------+
|2022|344433.39|
|2019|345618.83|
|2020|342693.17|
|2021|337899.58|
+----+---------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.

#what is this even asking


spark.sql("""
SELECT
YEAR(date) Year,
ROUND(AVG(price),2) Price
FROM homesales
WHERE  price = 350000 OR price > 350000
GROUP BY view
""").show()

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "date" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(date)" if you do not care which of the values within a group is returned.;
Aggregate [view#27], [year(date#18) AS Year#237, round(avg(price#20), 2) AS Price#238]
+- Filter ((price#20 = 350000) OR (price#20 > 350000))
   +- SubqueryAlias homesales
      +- View (`homesales`, [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27])
         +- Relation [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27] csv


In [13]:
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00010967254638671875 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homesales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('homesales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homesales_df.write.parquet('parquet_homesales', mode='overwrite')


homesales_df.write.partitionBy("date_built").mode("overwrite").parquet("homesales_partitioned")

In [23]:
# 11. Read the parquet formatted data.
parquet_df=spark.read.parquet('parquet_homesales')
parquet_df

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homesales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("homesales")
